# Synergies between SKA and SPHEREx

In [ ]:
#packages needed
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from configobj import ConfigObj
from scipy.interpolate import splrep, splev
import scipy as sp
import os

## Intro

We will use IM of 2 lines:
* HI: $\lambda=21$cm
* H$\alpha$: $\lambda=656.3$nm

We will use two surveys:
* SKA1 for HI
* SPHEREx for H$\alpha$

## Specs of each survey

### SKA1 in single dish mode

* Full sky
* Noise Power Spectrum: $\mathcal N^{ HI}_{ij}=\frac{S_{\rm area}}{2N_{\rm d} t_{\rm tot}}T^2_{\rm sys}\left(\nu\right)$ with $T_{\rm sys}=25+60(300\,{\rm MHz}/\nu)^{2.55}\,\rm K$. 
* ~200 dishes
* $N_d t_{\rm tot}=2\times10^6\,\rm hr$
* High frequency resolution
* Redshift range: z=0-3

### SPHEREx-like
* full sky
* Noise: $C_\ell^{\rm noise}=\sigma^2 (\nu I_\nu) \times \Omega_{\rm pixel}$
* $\sigma (\delta\nu I_\nu) \simeq 1 \times 10^{−17}$ erg/s/cm$^2$
* Angular Resolution: $\Omega_{\rm pixel}=6.2"\times6.2"=9.03\times 10^{-10}$Sr $\simeq 1 \times 10^{−9}$Sr
* Frequency resolution: $\lambda/\Delta\lambda= 41.5$ for $0.75<\lambda<4.1\mu$m
* Redshift range: z=0.1-5

## Spect for Multitracer

* Bins/Redshift Res: Chose $\delta z$, start at $z=0.2$. Example here for 0.1
* Sky coverage: $f_{\rm sky}=0.75$ (assumed). Sky dependence not studied.
* Angular pixel: In the case we have two intensity maps the pixel has to be the same for both. There is no gain of having one with an higher res than the other. Note that repixelizing SPHEREx, i.e., reducing the angular pixel does not change the noise power spectrum. And similarly to the radio. So we can use the pixel from the experiments consistentely. Changing the resolution does not affect the noise power spectrum, only the max available ell. 

In [ ]:
#Binning
z=np.arange(0.2,3,0.1) #chose binning

sigma_z=0.05
#top hat window with good smoothing
smooth=0.01

#Ha details. Link to file
ha_details=

z_Ha=np.loadtxt(ha_details)[:,0]
b_Ha_raw=np.loadtxt(ha_details)[:,2]
nuI_Ha_raw=np.loadtxt(ha_details)[:,1]
c=2.998e8
nu_ha=c/(656.3*1e-9*(1+z_Ha))
I_Ha_raw=nuI_Ha_raw/nu_ha

#HI details. Link to file
hi_details=
z_hi=np.loadtxt(hi_details)[:,0]
b_hi_raw=np.loadtxt(hi_details)[:,2]
T_hi_raw=np.loadtxt(hi_details)[:,3]

def from_raw_to_surv(z_raw,q_raw,new_z):
    rep=splrep(z_raw,q_raw)
    return splev(new_z,rep)
    
bHa=np.around(from_raw_to_surv(z_Ha,b_Ha_raw,z),3)
#in erg/s/cm^2/Hz/Sr
IHa=from_raw_to_surv(z_Ha,I_Ha_raw,z)
bHI=np.around(from_raw_to_surv(z_hi,b_hi_raw,z),3)
#in mK
THI=from_raw_to_surv(z_hi,T_hi_raw,z)

s_IM=0.4

## Numbers
nT=2
nw=len(z)


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(z,bHI,'b-',label=r'$b^{HI}_G$')
plt.plot(z,bHa,'r-',lw=3,label=r'$b^{H\alpha}_G$')
plt.plot(z,bHa/bHI,'k--',label=r'$b^{H\alpha}_G/b^{HI}_G$')
plt.xlabel(r'$z$',fontsize=14)
plt.ylabel(r'Bias',fontsize=14)
plt.legend(loc=2,fontsize=12,frameon=False)
plt.savefig('bias_HI_Halpha.pdf')
plt.show()
plt.plot(z,THI,label=r'$HI$')
plt.legend()
plt.show()
plt.plot(z,IHa,label=r'$H\alpha$')
plt.legend()
plt.show()

In [ ]:
#Write ini file
cs_folder='' #where template ini file of CAMB_sources is. In example file is called template.
file_isw_MT=''+'_root.ini' #name you want to give

os.system('cp '+cs_folder+'template.ini '+file_isw_MT)

inifile = ConfigObj(file_isw_MT)

inifile['output_root']= ''
inifile['l_max_scalar']= str(600)
inifile['accuracy_boost']= str(2)
inifile['l_accuracy_boost']= str(2)
inifile['l_sample_boost']= str(2)

inifile['counts_density']='T' 
inifile['counts_density_newt']='T' 
inifile['counts_redshift']= 'T'
inifile['DoRedshiftLensing']='F'
inifile['counts_radial']= 'F'
inifile['counts_timedelay']= 'F' 
inifile['counts_ISW']= 'T'
inifile['counts_velocity']= 'T'
inifile['counts_potential']= 'T' 
inifile['counts_evolve']= 'T'

inifile['num_redshiftwindows']=str(nw*nT)
for i in range(nw):
    #first HI Im
    inifile['redshift('+str(i+1)+')']= str(z[i])
    inifile['redshift_sigma('+str(i+1)+')']= str(sigma_z)
    inifile['redshift_kind('+str(i+1)+')']= 'counts'
    inifile['redshift_wintype('+str(i+1)+')']= 'smooth_tophat'
    inifile['redshift_smooth('+str(i+1)+')']= str(smooth)
    inifile['redshift_bias('+str(i+1)+')']= str(bHI[i])
    inifile['redshift_dlog10Ndm('+str(i+1)+')']= str(0.4)
    inifile['redshift_dNdz('+str(i+1)+')']= 'hiim'

for i in range(nw):
    #second Halpha IM
    inifile['redshift('+str(nw+i+1)+')']= str(z[i])
    inifile['redshift_sigma('+str(nw+i+1)+')']= str(sigma_z)
    inifile['redshift_kind('+str(nw+i+1)+')']= 'counts'
    inifile['redshift_wintype('+str(nw+i+1)+')']= 'smooth_tophat'
    inifile['redshift_smooth('+str(nw+i+1)+')']= str(smooth)
    inifile['redshift_bias('+str(nw+i+1)+')']= str(bHa[i])
    inifile['redshift_dlog10Ndm('+str(nw+i+1)+')']= str(0.4)
    inifile['redshift_dNdz('+str(nw+i+1)+')']= 'halpha'
    
inifile.write()

## Noise of each survey

### H$\alpha$ with Spherex

In [ ]:
pix_spherex=1e-9 #Sr
#flux sensitivity
sign_dnuI_spherex=1e-17 #erg/s/cm^2
#per Hz
dnu_spherex=nu_ha/41.5
#frequency size of the bin
Delta_nu_bin_halpha=1e9*c/(656.3)*(1/(1+z-sigma_z)-1/(1+z+sigma_z))
#the noise C_\ell
cl_noise_spherex=(sign_dnuI_spherex)**2/pix_spherex/from_raw_to_surv(z_Ha,dnu_spherex,z)/Delta_nu_bin_halpha #erg^2/s^2/cm^4/ Sr


### HI with SKA1

In [ ]:
Delta_nu_bin_hi=1420.4*1e6*(1/(1+z-sigma_z)-1/(1+z+sigma_z))
nu_hi=1420.4/(1+z)#Mhz
T2_sys=((25 + 60*(300/nu_hi)**2.55)*1e3)**2 #mK^2
Ndttot = 2e6*3600 #s
N_IM_no_sky=2.0*np.pi*T2_sys/(Ndttot*Delta_nu_bin_hi)

## Normalizations

In [ ]:
## Normalization
I_Ha_bin=IHa
print(I_Ha_bin)
Norm=np.ones((nT*nw,nT*nw))
Norm[0:nw,0:nw]=np.array([THI]).T*THI
Norm[0:nw,nw:]=np.array([THI]).T*I_Ha_bin
Norm[nw:,0:nw]=Norm[0:nw,nw:].T
Norm[nw:,nw:]=np.array([I_Ha_bin]).T*I_Ha_bin

In [ ]:
plt.plot(z,I_Ha_bin**2/cl_noise_spherex,'ko')
plt.plot(z,THI**2/(N_IM_no_sky*0.75),'b*')
plt.yscale('log')

## Parameters in the fisher matrix

The natural ones would be what we want to measure or diferentiate
* $f_{\rm NL}$
* $\epsilon_{\rm Doppler}$
* $\epsilon_{\rm SW}$
* $\epsilon_{\rm ISW}$

The others would be the normal cosmological parameters $\vartheta={\ln H_0,\ln A_s,\ln n_s,\ln \Omega_{CDM},\ln \Omega_{b},w}$. We will marginalise over all the bias.

The fiducial are $H_0=67,74$ km/s/Mpc, $A_s=2.142 \times 10^{−9}$, $n_s=0.967$, $\Omega_{CDM}=0.26$,  $\Omega_{b}=0.05$,$w=-1$, $f_{\rm NL}=0$, $\epsilon_{\rm GR}=1$ and $\epsilon_{\rm ISW}=1$.


## Load the angular power spectra

In [ ]:
folder='' #folder where the files are
root=''  #root name of all files needed for this bin
parametername=['Ocdm','ns','Ob','w','H0','fnl','eISW','edoppler','eSW']
for i in range(0,nw):
    parametername=parametername+['bhi'+str(i+1)]
for i in range(0,nw):
    parametername=parametername+['bha'+str(i+1)]
print(parametername)

fid_norm=[0.26,0.9667,0.05,1,67.74,1.27,1,1,1]+[1]*nT*nw
#fnl multiplied by 1.27 for the CMB convention
pars=['As']+parametername

In [ ]:
var_fisher={}
### Cut the first bin

var_fisher['l']=np.arange(2,501)
var_fisher['fid']=np.loadtxt(folder+root+'fid_Cl.dat')[:,1:]
var_fisher['As']=var_fisher['fid']
for i in range(len(parametername)):
    var_fisher[parametername[i]]=np.loadtxt(folder+root+parametername[i]+'_dCl.dat')[:,1:]*fid_norm[i]

klass = type('Cls', (object,), var_fisher)
Cls_ISW_HI_Halpha = klass()

## Define and Compute the Fisher Matrix

In [ ]:

def H(z):
    return 67.74*(0.31*(1+z)**3+0.69)**0.5
#in Km/s/Mpc

def chi(z):
    chiev=np.zeros(len(z))
    for i in range(len(z)):
        zinte=np.arange(0,z[i],0.001)
        chiev[i]=np.trapz(c*1e-3/H(zinte),zinte)
    return chiev
#Max ell that is linear
ell_bin_max=np.append(np.around(0.2*0.67*(1+z)**(2/(2+0.967))*chi(z),0),
                   np.around(0.2*0.67*(1+z)**(2/(2+0.967))*chi(z),0))



In [ ]:
def fisher_fsky_l(Cls_mtr,fsky,lmin,lmax,norm,N_IM_no_sky,cl_noise_spherex,ell_bin_max,pars,windows,
                  tracers,noise_level=1,use_cut=False,cut=[]):
    bins=windows*tracers
    cut_or=np.arange(0,bins)
    npars=len(pars)
    fisher=np.zeros((npars,npars))
    if lmin<2:
        lmin=2
    l_range=np.arange(lmin-2,lmax-1,1)
    Noise=np.zeros((windows*tracers,windows*tracers))    
    Noise[0:windows,0:windows]=np.diag(N_IM_no_sky*fsky)
    Noise[windows:,windows:]=np.diag(cl_noise_spherex)
    
    
    for k in l_range:
        if use_cut==True:
            cut=cut
        else:
            cut_index=(ell_bin_max>k)
            cut=cut_or[cut_index]
        temp=np.zeros((npars,npars))
        #eigvals, Umatrix=sp.linalg.eigh((Cls_mtr.fid[k,:].reshape(bins,bins)*norm+Noise*noise_level)[cut,:][:,cut])
        #Triag, Z=sp.linalg.schur((Cls_mtr.fid[k,:].reshape(bins,bins)*norm+Noise*noise_level)[cut,:][:,cut])
        invGamma=sp.linalg.inv((Cls_mtr.fid[k,:].reshape(bins,bins)*norm+Noise*noise_level)[cut,:][:,cut])
        #invGamma=np.dot(np.dot(Umatrix,np.diag(1/eigvals)),sp.linalg.inv(Umatrix))
        #invGamma=np.dot(Z,np.dot(sp.linalg.inv(Triag),Z.T))
        #all methods are equivalent
        for i in range(npars):
            dcl1=((getattr(Cls_mtr, pars[i])[k,:].reshape(bins,bins)*norm))[cut,:][:,cut]
            for j in range(npars):
                if j<i:
                    next                  
                dcl2=((getattr(Cls_mtr, pars[j])[k,:].reshape(bins,bins)*norm))[cut,:][:,cut]
                temp[i,j]=np.trace(dcl1.dot(invGamma.dot(dcl2.dot(invGamma))))
                temp[j,i]=temp[i,j]
        
        fisher=fisher+(2*Cls_mtr.l[k]+1)*temp
    
    fisher=fisher*fsky/2.0
    return fisher


In [ ]:
# Sky overlaps
f_sky=[0.5,0.75]

## Single tracer

### SKA only


In [ ]:
for fsky in f_sky:
    print('Sky Area:', fsky)
    lmin_obs=int(np.pi/np.sqrt(4*np.pi*fsky))+1
    fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,fsky,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,pars[:-nw],nw,2,noise_level=1,use_cut=True,cut=np.arange(0,nw))
    sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)
    print('sigma :','Marginal','Conditional')
    for i in range(len(pars[:-nw])):
        sig_epsISW_IM_HI_Ha=np.sqrt(sigma2_IM_HI_Ha[i,i])
        print(pars[i]+':',sig_epsISW_IM_HI_Ha,',',np.sqrt(1/fisher_IM_HI_Ha[i,i]))

## SPHEREx only

In [ ]:
par_Ha=pars[:-(2*nw)]+pars[-nw:]
for fsky in f_sky:
    print('Sky Area:', fsky)
    lmin_obs=int(np.pi/np.sqrt(4*np.pi*fsky))+1
    fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,fsky,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,par_Ha,nw,2,use_cut=True,cut=np.arange(nw,2*nw))
    sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)
    print('sigma :','Marginal','Conditional')
    for i in range(len(par_Ha)):
        sig_epsISW_IM_HI_Ha=np.sqrt(sigma2_IM_HI_Ha[i,i])
        print(par_Ha[i]+':',sig_epsISW_IM_HI_Ha,',',np.sqrt(1/fisher_IM_HI_Ha[i,i]))

## MT SKA SPHEREx

In [ ]:
for fsky in f_sky:
    print('Sky Area:', fsky)
    lmin_obs=int(np.pi/np.sqrt(4*np.pi*fsky))+1
    fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,fsky,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,pars,nw,2,noise_level=1)
    sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)#+prior)
    print('sigma :','Marginal','Conditional')
    for i in range(len(pars)):
        sig_epsISW_IM_HI_Ha=np.sqrt(sigma2_IM_HI_Ha[i,i])
        print(pars[i]+':',sig_epsISW_IM_HI_Ha,',',np.sqrt(1/fisher_IM_HI_Ha[i,i]))

In [ ]:
#noiseless regime
for fsky in f_sky:
    print('Sky Area:', fsky)
    lmin_obs=int(np.pi/np.sqrt(4*np.pi*fsky))+1
    fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,fsky,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,pars,nw,2,noise_level=0)
    sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)#+prior)
    print('sigma :','Marginal','Conditional')
    for i in range(len(pars)):
        sig_epsISW_IM_HI_Ha=np.sqrt(sigma2_IM_HI_Ha[i,i])
        print(pars[i]+':',sig_epsISW_IM_HI_Ha,',',np.sqrt(1/fisher_IM_HI_Ha[i,i]))

## Effect of varying the noise 

In [ ]:
pars_gr=[r'$f_{\rm NL}$',r'$\varepsilon_{\rm ISW}$',r'$\varepsilon_{\rm Doppler}$',r'$\varepsilon_{\rm SW}$']
lmin_obs=int(np.pi/np.sqrt(4*np.pi*0.5))+1
noise_per=np.arange(-4,1.1,0.2)


In [ ]:
sigmas_75=np.zeros((5,len(noise_per)))
for i in range(len(noise_per)):
    print(noise_per[i])
    fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,0.75,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,pars,nw,2,noise_level=10**noise_per[i])
    sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)
    sigmas_75[:,i]=np.sqrt(np.diagonal(sigma2_IM_HI_Ha[6:11,6:11]))
    
np.savetxt('gr_pars_noise_dz01.dat',sigmas_75)

In [ ]:
pars_gr=[r'$f_{\rm NL}$',r'$\varepsilon_{\rm ISW}$',r'$\varepsilon_{\rm Doppler}$',r'$\varepsilon_{\rm SW}$']
colors='mgbr'
style=[':','-.','-','--']
size_line=[4,3,2,1]
plt.figure(figsize=(6,4))
for i in range(4):
    plt.plot(10**noise_per,sigmas_75[i,:],color=colors[i],ls=style[i],lw=size_line[i],label=pars_gr[i])
plt.axhline(y=1, xmin=0.01, xmax=1, linewidth=0.5, color = 'k')
plt.xlabel('Noise Level %',fontsize=14)
plt.ylabel(r'$\sigma$',fontsize=14)
plt.xscale('log')
plt.yscale('log')
plt.ylim([4e-3,12])
plt.legend(loc=4,frameon=False,fontsize=14,ncol=2)
plt.savefig('noise_level_effect_dz01_fsky75.pdf')
plt.show()

### Make contour plots

In [ ]:
fsky=0.75
lmin_obs=int(np.pi/np.sqrt(4*np.pi*fsky))+1
fisher_IM_HI_Ha=fisher_fsky_l(Cls_ISW_HI_Halpha,0.75,lmin_obs,300,Norm,N_IM_no_sky,cl_noise_spherex,
                              ell_bin_max,pars,nw,2)
sigma2_IM_HI_Ha=np.linalg.inv(fisher_IM_HI_Ha)
sigma2_IM_HI_Ha

In [ ]:
fnl_array=np.arange(-6,6,0.05)
eps_array=np.arange(-15,16,0.05)

def confidence(x,y,xfid,yfid,F):
    #sig2=sp.linalg.det(sp.linalg.inv(F))
    pararray=np.array([x,y])
    fidarray=np.array([xfid,yfid])
    test_vec=pararray-fidarray
    #return 1/(2*np.pi*sig2)*np.exp(-(test_vec.dot(F.dot(test_vec.T)))/2)
    return test_vec.dot(F.dot(test_vec.T))

In [ ]:
contour_doppler=np.zeros((len(fnl_array),len(eps_array)))
contour_isw=np.zeros((len(fnl_array),len(eps_array)))
contour_pot=np.zeros((len(fnl_array),len(eps_array)))


for i in range(len(fnl_array)):
    for j in range(len(eps_array)):
        contour_isw[i,j]=confidence(fnl_array[i],eps_array[j],0,1,np.linalg.inv(sigma2_IM_HI_Ha[[6,7],:][:,[6,7]]))
        contour_pot[i,j]=confidence(fnl_array[i],eps_array[j],0,1,np.linalg.inv(sigma2_IM_HI_Ha[[6,9],:][:,[6,9]]))
        contour_doppler[i,j]=confidence(fnl_array[i],eps_array[j],0,1,np.linalg.inv(sigma2_IM_HI_Ha[[6,8],:][:,[6,8]]))
        
    

In [ ]:
plt.figure()
plt.contour(fnl_array, eps_array,contour_isw.T,colors='g',levels=[2.3,6.17],linewidths=(3,1),linestyles='-')
plt.contour(fnl_array, eps_array,contour_doppler.T,colors='b',levels=[2.3,6.17],linewidths=(3,1),linestyles='-.')
plt.contour(fnl_array, eps_array,contour_pot.T,colors='r',levels=[2.3,6.17],linewidths=(3,1),linestyles='--')
plt.xlabel(r'$f_{\rm NL}$',fontsize=14)
plt.ylabel(r'$\varepsilon$',fontsize=14)
plt.plot(0,1,'k+',ms=10)
cores='gbr'
import matplotlib.lines as mlines
eps_line=[]
style=['-','-.','--']
for i in range(3):
    eps_line.append(mlines.Line2D([], [], color=cores[i],label=pars_gr[i+1],linestyle=style[i],lw=3))
    #plt.legend(handles=[eps_line],loc=2)
first_legend=plt.legend(handles=eps_line[:],loc=8,fontsize=16,ncol=3)
plt.gca().add_artist(first_legend)
sig_line=[]
sig_label=[r'$1-\sigma$',r'$2-\sigma$']
for i in range(2):
    sig_line.append(mlines.Line2D([], [], color='k',label=sig_label[i],linewidth=[3,1][i]))
plt.legend(handles=sig_line[:],loc=9,frameon=False,fontsize=12,ncol=2)
plt.ylim([-4,5])
plt.xlim([-3.5,3.5])
plt.savefig('onesigma_fnl_eps_dz01_compaxis.pdf')
plt.show()